#Levels

- Level 1: The best bid, ask and auction info
- Level 2: Full order book (aggregated) and auction info
- Level 3: Full order book (non aggregated) and auction info

# Understanding the data
- asks: [['19106.3', '0.255', 2]]
  - price at that time: 19106.3
  - size(is the sum of the size of the orders at that price): 0.255
  - num-orders: 2

#importing the necessary libraries

In [ ]:
!pip install schedule
!pip install cbpro

import cbpro
import pandas as pd
c = cbpro.PublicClient()

import schedule
import time , datetime

In [33]:

order_book_btc = c.get_product_order_book('BTC-USD',level=2)
order_book_eth = c.get_product_order_book('BTC-USD',level=2)

#funtion to load bitcoin asks data into a dataframe
def get_btc_asks_data():
  order_book_btc = c.get_product_order_book('BTC-USD',level=2)
  ask = pd.DataFrame(order_book_btc['asks'], columns=('price','size','num_orders'))
  ask.to_csv('ask-btc.csv', encoding='utf-8', index=False)
  return ask
  #print(ask)


# schedule.every(60).seconds.do(get_btc_usd_asks_data)
# while True:
#   schedule.run_pending()
#   #time.sleep(60 - datetime.datetime.now().second)
#   #time.sleep(1)


#funtion to load bitcoin bids data into a dataframe
def get_btc_bids_data():
  order_book_btc = c.get_product_order_book('BTC-USD',level=2)
  ask = pd.DataFrame(order_book_btc['bids'], columns=('price','size','num_orders'))
  ask.to_csv('bids-btc.csv', encoding='utf-8', index=False)
  return ask

In [34]:
#funtion to load ETH bid data into a dataframe
def get_eth_asks_data():
  order_book_eth = c.get_product_order_book('BTC-USD',level=2)
  ask = pd.DataFrame(order_book_eth['asks'], columns=('price','size','num_orders'))
  ask.to_csv('ask.eth', encoding='utf-8', index=False)
  return ask

  #funtion to load bitcoin bid data into a dataframe
def get_eth_bids_data():
  order_book_eth = c.get_product_order_book('BTC-USD',level=2)
  ask = pd.DataFrame(order_book_eth['asks'], columns=('price','size','num_orders'))
  ask.to_csv('bids.eth', encoding='utf-8', index=False)
  return ask

Creating Connection go Google BigQuery API

In [6]:
!pip install --upgrade google-cloud-bigquery
from google.cloud import bigquery
import pandas

from google.cloud import bigquery
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
'/content/google_service_account_key.json')

project_id = 'prefab-grid-328715'
client = bigquery.Client(credentials= credentials,project=project_id)

#loading data into BigQuery staging Datasets

In [35]:
#client = bigquery.Client()
table_id_ask_eth_usd = "prefab-grid-328715.COINBASE_DATA_ETH_USD_STG.ASK_ETH_USD"
table_id_bids_eth_usd = "prefab-grid-328715.COINBASE_DATA_ETH_USD_STG.BIDS_ETH_USD"
table_id_bids_btc_usd = "prefab-grid-328715.COINBASE_DATA_BTC_USD_STG.BIDS_BTC_USD"
table_id_ask_btc_usd = "prefab-grid-328715.COINBASE_DATA_BTC_USD_STG.ASK_BTC_USD"


dataframe1 = pandas.DataFrame(
   get_btc_asks_data(),


    columns = [
           "price",
           "size",
           "num_orders"
]

)

dataframe2 = pandas.DataFrame(
    get_btc_bids_data(),

    columns = [
           "price",
           "size",
           "num_orders"
]

)
dataframe3 = pandas.DataFrame(
    get_eth_asks_data(),
    

    columns = [
           "price",
           "size",
           "num_orders"
]

)
dataframe4 = pandas.DataFrame(
    get_eth_bids_data(),
   

    columns = [
           "price",
           "size",
           "num_orders"
]

)

job_config = bigquery.LoadJobConfig(
    schema = [
        bigquery.SchemaField("price", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("size", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("num_orders", bigquery.enums.SqlTypeNames.INT64) 
  ]
)


#jobs to load data 
#Make an API request
job1 = client.load_table_from_dataframe(dataframe1, table_id_ask_btc_usd, 
                                job_config=job_config)

job2 = client.load_table_from_dataframe(dataframe2, table_id_bids_btc_usd, 
                                job_config=job_config)

job3 = client.load_table_from_dataframe(dataframe3, table_id_ask_eth_usd, 
                                job_config=job_config)

job4 = client.load_table_from_dataframe(dataframe4,table_id_bids_eth_usd , 
                                job_config=job_config)
 #wait for the job to complete
job1.result()
job2.result()
job3.result()
job4.result()

table1 = client.get_table(table_id_ask_btc_usd)
table2 = client.get_table(table_id_bids_btc_usd)
table3 = client.get_table(table_id_ask_eth_usd)
table4 = client.get_table(table_id_bids_eth_usd)

print(table1)
print(table2)
print(table3)
print(table4)


Table(TableReference(DatasetReference('prefab-grid-328715', 'COINBASE_DATA_BTC_USD_STG'), 'ASK_BTC_USD'))
Table(TableReference(DatasetReference('prefab-grid-328715', 'COINBASE_DATA_BTC_USD_STG'), 'BIDS_BTC_USD'))
Table(TableReference(DatasetReference('prefab-grid-328715', 'COINBASE_DATA_ETH_USD_STG'), 'ASK_ETH_USD'))
Table(TableReference(DatasetReference('prefab-grid-328715', 'COINBASE_DATA_ETH_USD_STG'), 'BIDS_ETH_USD'))


#load data from stage datasets to production 

In [36]:
#loading btc_ask_data
def load_data1():
  query_job = client.query(
    """
    
INSERT INTO `prefab-grid-328715.COINBASE_DATA_BTC_USD.ASK_BTC_USD` 
(PRICE, SIZE , NUM_ORDERS )   

SELECT  
CAST(PRICE as NUMERIC) as PRICE
,CAST (SIZE as NUMERIC) as SIZE
,num_orders as NUM_ORDERS
 FROM `prefab-grid-328715.COINBASE_DATA_BTC_USD_STG.ASK_BTC_USD`  """
)
  results = query_job.result()





In [37]:

#loading btc_bids_data
def load_data2():
  query_job = client.query(
    """
    
    
INSERT INTO `prefab-grid-328715.COINBASE_DATA_BTC_USD.BIDS_BTC_USD`
(PRICE, SIZE , NUM_ORDERS )   

SELECT  
CAST(PRICE as NUMERIC) as PRICE
,CAST (SIZE as NUMERIC) as SIZE
,num_orders as NUM_ORDERS
 FROM `prefab-grid-328715.COINBASE_DATA_BTC_USD_STG.BIDS_BTC_USD`  """
)
  results = query_job.result()

In [38]:
#loading eth_ask_data
def load_data3():
  query_job = client.query(
    """
    
INSERT INTO `prefab-grid-328715.COINBASE_DATA_ETH_USD.ASK_ETH_USD ` 
(PRICE, SIZE , NUM_ORDERS )   

SELECT  
CAST(PRICE as NUMERIC) as PRICE
,CAST (SIZE as NUMERIC) as SIZE
,num_orders as NUM_ORDERS
 FROM `prefab-grid-328715.COINBASE_DATA_ETH_USD_STG.ASK_ETH_USD`   """
)
  results = query_job.result()

In [39]:
#loading eth_bids_data
def load_data4():
  query_job = client.query(
    """
    
INSERT INTO `prefab-grid-328715.COINBASE_DATA_ETH_USD.BIDS_ETH_USD ` 
(PRICE, SIZE , NUM_ORDERS )   

SELECT  
CAST(PRICE as NUMERIC) as PRICE
,CAST (SIZE as NUMERIC) as SIZE
,num_orders as NUM_ORDERS
 FROM `prefab-grid-328715.COINBASE_DATA_ETH_USD_STG.BIDS_ETH_USD`   """
)
  results = query_job.result()

In [40]:
#execute loads
load_data1()
load_data2()
load_data3()
load_data4()

#use full links

https://datascience.stackexchange.com/questions/72483/how-to-extract-data-from-an-api-every-hour-in-python
https://curatedpython.com/p/coinbasepro-python-danpaquin-coinbasepro-python/index.html
https://docs.cloud.coinbase.com/sign-in-with-coinbase/docs/client-libraries
https://algotrading101.com/learn/coinbase-pro-api-guide/